In [ ]:
!pip install -r requirements.txt

In [ ]:
import argparse
import multiprocessing
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
from xglue.utils import *
from xglue.model import *
cpu_count = 4

In [ ]:
args = argparse.Namespace()
argsdict = vars(args)
argsdict["train_data_file"] = "train.txt"
argsdict["output_dir"] = "data/code-clone/saved_models"
argsdict["eval_data_file"] = "valid.txt"
argsdict["test_data_file"] = "test.txt"
argsdict["model_type"] = "roberta"
argsdict["model_name_or_path"] = "microsoft/codebert-base"
argsdict["tokenizer_name"] = "roberta-base"
argsdict["config_name"] = "microsoft/codebert-base"
argsdict["cache_dir"] = ""

argsdict["mlm"] = False
argsdict["mlm_probability"] = 0.15
argsdict["block_size"] = 200
argsdict["do_train"] = True
argsdict["do_eval"] = False
argsdict["do_test"] = False
argsdict["evaluate_during_training"] = True
argsdict["do_lower_case"] = False
argsdict["train_batch_size"] = 16
argsdict["eval_batch_size"] = 32
argsdict["gradient_accumulation_steps"] = 1
argsdict["learning_rate"] = 5e-5
argsdict["weight_decay"] = 0.0
argsdict["adam_epsilon"] = 1e-8
argsdict["max_grad_norm"] = 1.0
argsdict["num_train_epochs"] = 1.0
argsdict["max_steps"] = -1
argsdict["warmup_steps"] = 0
argsdict["logging_steps"] = 2
argsdict["save_steps"] = 2
argsdict["save_total_limit"] = None
argsdict["eval_all_checkpoints"] = False
argsdict["no_cuda"] = False
argsdict["overwrite_output_dir"] = False
argsdict["overwrite_cache"] = True
argsdict["seed"] = 123456
argsdict["epoch"] = 1
argsdict["fp16"] = False
argsdict["fp16_opt_level"] = '01'
argsdict["local_rank"] = -1
argsdict["server_ip"] = ''
argsdict["server_port"] = ''
argsdict["per_gpu_train_batch_size"] = 16
argsdict["n_gpu"] = 1
argsdict["device"] = "cpu"
#
args.start_epoch = 0
args.start_step = 0

In [ ]:
pool = multiprocessing.Pool(cpu_count)
tokenizer = RobertaTokenizer.from_pretrained(args.tokenizer_name,
                                                do_lower_case=False,
                                                cache_dir=args.cache_dir if args.cache_dir else None,
                                             model_max_length=512, max_length=512, padding="max_length", truncation=True)


In [ ]:
config = RobertaConfig.from_pretrained(args.config_name if args.config_name else args.model_name_or_path,
                                          cache_dir=args.cache_dir if args.cache_dir else None, model_max_length=512)
config.num_labels=1

model = RobertaModel.from_pretrained(args.model_name_or_path,
                                            from_tf=bool('.ckpt' in args.model_name_or_path),
                                            config=config,
                                            cache_dir=args.cache_dir if args.cache_dir else None)

In [ ]:
train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False,pool=pool)

In [ ]:
model=Model(model,config,tokenizer,args)
global_step, tr_loss = train(args, train_dataset, model, tokenizer, pool)